# validation Graph - DiGraph

In [1]:
'''import os
import sys
new_path = os.getcwd()[:-26] + 'qualicharge-rtet'
sys.path.append(new_path)'''

"import os\nimport sys\nnew_path = os.getcwd()[:-26] + 'qualicharge-rtet'\nsys.path.append(new_path)"

In [2]:
import geopandas as gpd
import pandas as pd
import geo_nx as gnx 
import networkx as nx 

from qualicharge_rtet import troncons_non_mailles, troncons_peu_mailles, gr_maillage
from qualicharge_rtet import to_macro_node

rtet_path = '../rtet/'
refnat = {'tiles': 'cartodbpositron', 'location': [46.3, 2.3], 'zoom_start': 7}
vl_central_2025 = (True, 400, 150, 60)
vl_central_2027 = (True, 600, 150, 60)
vl_global_2027 = (False, 300, 150, 60)
pl_global_2025 = (False, 1400, 350, 120)
pl_central_2027 = (True, 2800, 350, 120)


version = 'V1'
scenario = vl_global_2027

## Chargement AFIR



In [3]:
central, pcum, pmax, dist = scenario
add_name = '_core' if central else ''
add_pcum = '_' + str(pcum)    
add_pmax = '_' + str(pmax)

gr_tot = gnx.from_geopandas_edgelist(gpd.read_file(rtet_path + version + add_name + add_pcum + add_pmax + '_afir_edge.geojson'), 
                                     node_gdf=gpd.read_file(rtet_path + version + add_name + add_pcum + add_pmax + '_afir_node.geojson'),
                                     node_attr=True, edge_attr=True)
#gr_tot.clean_attributes()

In [4]:
len(gr_tot.nodes), len(gr_tot.edges)

(3713, 3786)

In [14]:
#for edge in gr_tot.edges:
#    print(gr_tot.edges[edge])

In [6]:
dgr_tot = gr_tot.to_directed()
def node_autoroute(node):
    for pred_node in dgr_tot.predecessors(node):
        #print(dgr_tot.edges[pred_node, node])
        if dgr_tot.edges[pred_node, node]['nature'] != 'troncon autoroute':
            return False
    for succ_node in dgr_tot.successors(node):
        if dgr_tot.edges[node, succ_node]['nature'] != 'troncon autoroute':
            return False
    return True

nodes = [node for node in dgr_tot.nodes if dgr_tot.nodes[node]['nature'] == 'connecteur' and 
        node_autoroute(node)]
print(len(nodes))
for node in nodes:
    to_macro_node(dgr_tot, node)

108


In [7]:
stat_edges = [edge for edge in gr_tot.edges if gr_tot.edges[edge]['nature'][:7] == 'liaison']
gs = nx.edge_subgraph(gr_tot, stat_edges)
rtet_edges = [edge for edge in gr_tot.edges if gr_tot.edges[edge]['nature'][:7] != 'liaison']
gr = nx.edge_subgraph(gr_tot, rtet_edges)

In [8]:
#for edge in dgr_tot.edges:
#    print(dgr_tot.edges[edge])

In [9]:
dstat_edges = [edge for edge in dgr_tot.edges if dgr_tot.edges[edge]['nature'][:7] == 'liaison']
dgs = nx.edge_subgraph(dgr_tot, dstat_edges)
drtet_edges = [edge for edge in dgr_tot.edges if dgr_tot.edges[edge]['nature'][:7] != 'liaison']
dgr = nx.edge_subgraph(dgr_tot, drtet_edges)

### Trajet entre deux stations
Exemple entre la station '4128' (Bollene) et la station '4148' (saint-martin de crau)

In [10]:
bollene = [node for node in gr_tot.nodes if gr_tot.nodes[node].get('id_station') == 'FR*IZF*EFAST*95*1*3']
bollene = bollene[0] if bollene else None
st_martin_de_crau = [node for node in gr_tot.nodes if gr_tot.nodes[node].get('id_station') == 'FR*PVD*EVG*SMC13*D01*1']
st_martin_de_crau = st_martin_de_crau[0] if st_martin_de_crau else None
trajet = bollene and st_martin_de_crau
print('trajet : ', bollene, st_martin_de_crau)

trajet :  27965 28271


In [11]:
bollene = [node for node in dgr_tot.nodes if dgr_tot.nodes[node].get('id_station') == 'FR*IZF*EFAST*95*1*3']
bollene = bollene[0] if bollene else None
st_martin_de_crau = [node for node in dgr_tot.nodes if dgr_tot.nodes[node].get('id_station') == 'FR*PVD*EVG*SMC13*D01*1']
st_martin_de_crau = st_martin_de_crau[0] if st_martin_de_crau else None
trajet = bollene and st_martin_de_crau
print('trajet : ', bollene, st_martin_de_crau)

trajet :  27965 28271


In [12]:
if trajet:
    print('distance entre les stations : ',
    nx.shortest_path_length(gr_tot, source=bollene, target=st_martin_de_crau, weight='weight'))

distance entre les stations :  116138.54862765616


In [13]:
if trajet:
    print('distance entre les stations : ',
    nx.shortest_path_length(dgr_tot, source=bollene, target=st_martin_de_crau, weight='weight'))

NetworkXNoPath: Node 28271 not reachable from 27965

In [ ]:
if trajet:
    path = nx.shortest_path(gr_tot, source=bollene, target=st_martin_de_crau, weight='weight')
    gr_path = nx.subgraph_view(gr_tot, filter_edge=(lambda x, y: x in path and y in path and abs(path.index(y) - path.index(x)) == 1), filter_node=lambda x: x in path)
    print(path, type(path), nx.path_weight(gr_tot, path, 'weight'))

In [ ]:
if trajet:
    path = nx.shortest_path(dgr_tot, source=bollene, target=st_martin_de_crau, weight='weight')
    dgr_path = nx.subgraph_view(dgr_tot, filter_edge=(lambda x, y: x in path and y in path and abs(path.index(y) - path.index(x)) == 1), filter_node=lambda x: x in path)
    print(path, type(path), nx.path_weight(gr_tot, path, 'weight'))

In [ ]:
param_exp_gs = {'e_tooltip': ["source", "target"], 'e_popup': ['nature', 'weight', 'source', 'target'], 'e_color': 'grey',
                'n_name': 'station', 'e_name': 'liaison station', 'n_popup': ['amenageur', 'operateur', 'p_cum', 'p_max', 'node_id', 'id_station'], 
                'n_tooltip': "amenageur", 'n_color': 'green', 'n_marker_kwds': {'radius': 4, 'fill': True}}

param_exp_gr = {'e_popup': ['weight', 'source', 'target', 'core', "CORRIDORS", "INTROADNUM", "NATIONALRO", "ID", "nature"], 
                'n_popup': ['roadname', 'nom', 'node_id'],
                'e_tooltip': ["source", "target", "nature", "core"], 
                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
#param_exp_gr = {'e_name': 'edges', 'n_name': 'nodes', 'e_popup': ['weight', 'source', 'target'], 'e_tooltip': ["source", "target"], 
#                'n_tooltip': ["node_id", "nature"], 'n_marker_kwds': {'radius': 1, 'fill': False}}
param_exp_path = {'e_name': 'path', 'e_color': 'red'}
refmap = {'tiles': 'cartodbpositron', 'location': [43.8, 5], 'zoom_start': 9}

if trajet:    
    carte = gs.explore(refmap=refmap, **param_exp_gs)
    carte = gr.explore(refmap=carte, n_color='black', e_name='edges rtet', n_name='nodes rtet', **param_exp_gr)
    carte = gr_path.explore(refmap=carte, layercontrol=True, nodes=False, **param_exp_path)
carte

In [ ]:
if trajet:    
    carte = dgs.explore(refmap=refmap, **param_exp_gs)
    carte = dgr.explore(refmap=carte, n_color='black', e_name='edges rtet', n_name='nodes rtet', **param_exp_gr)
    carte = dgr_path.explore(refmap=carte, layercontrol=True, nodes=False, **param_exp_path)
carte

### Tronçons avec un faible maillage (méthode par tronçon)

Identification des tronçons avec maillage insuffisant:
- un tronçon est non maillé si en tout point du tronçon, la distance entre les deux stations les plus proches est supérieure à un seuil,
- un tronçon est peu maillé s'il est sur la même branche (ensemble de tronçons entre deux stations disponibles ou bifurcations) qu'un tronçon maillé.

La station '4501' (gap) est considéré pour cet exemple comme indisponible.

In [ ]:
sub_auto = nx.subgraph_view(gr, filter_edge=lambda x,y: gr[x][y]["nature"] == 'troncon autoroute')
sub_hors_auto = nx.subgraph_view(gr, filter_edge=lambda x,y: gr[x][y]["nature"] == 'troncon hors autoroute')

gr_autoroute =  nx.induced_subgraph(sub_auto, nx.utils.flatten(sub_auto.edges()))
gr_hors_autoroute =  nx.induced_subgraph(sub_hors_auto, nx.utils.flatten(sub_hors_auto.edges()))

In [ ]:
sub_auto = nx.subgraph_view(dgr, filter_edge=lambda x,y: dgr[x][y]["nature"] == 'troncon autoroute')
sub_hors_auto = nx.subgraph_view(dgr, filter_edge=lambda x,y: dgr[x][y]["nature"] == 'troncon hors autoroute')

dgr_autoroute =  nx.induced_subgraph(sub_auto, nx.utils.flatten(sub_auto.edges()))
dgr_hors_autoroute =  nx.induced_subgraph(sub_hors_auto, nx.utils.flatten(sub_hors_auto.edges()))

In [ ]:
seuil = dist * 1000
dispo = 'dispo'

test_saturation = False
if test_saturation:
    gr_tot.add_node(4501, dispo=False) # indisponibilité d'une station (Gap)

from time import time
t0 = time()
gr_stat_indispo, gr_non_maille = troncons_non_mailles(gr_tot, gs, gr, dispo, seuil)
print(time()-t0)
gr_peu_maille = troncons_peu_mailles(gr_non_maille, gr_tot, dispo)

In [ ]:
seuil = dist * 1000
dispo = 'dispo'
if test_saturation:
    dgr_tot.add_node(4501, dispo=False) # indisponibilité d'une station (Gap)

from time import time
t0 = time()
dgr_stat_indispo, dgr_non_maille = troncons_non_mailles(dgr_tot, dgs, dgr, dispo, seuil)
print(time()-t0)
dgr_peu_maille = troncons_peu_mailles(dgr_non_maille, dgr_tot, dispo)

In [ ]:
#refnat |= {'tiles': 'openstreetmap'}
#refnat |= {'tiles': 'cartodbdark_matter'}
refnat |= {'tiles': None}

param_non_maille = {'e_name': 'non maillé', 'e_color': 'red', 'n_color': 'red'}
param_peu_maille = {'e_name': 'peu maillé', 'e_color': 'orange'}

carte = gs.explore(refmap=refnat, **param_exp_gs)
carte = gr_hors_autoroute.explore(refmap=carte, e_color='green', e_name='edges hors autoroute', n_name='nodes hors autoroute', n_color='black', **param_exp_gr)
carte = gr_autoroute.explore(refmap=carte, n_color='black', e_name='edges autoroute', n_name='nodes autoroute', **param_exp_gr)
carte = gr_peu_maille.explore(refmap=carte, **param_peu_maille, nodes=False)
carte = gr_stat_indispo.explore(refmap=carte, **param_non_maille)
carte = gr_non_maille.explore(refmap=carte, layercontrol=True, nodes=False, **param_non_maille)
carte

In [ ]:
carte = dgs.explore(refmap=refnat, **param_exp_gs)
carte = dgr_hors_autoroute.explore(refmap=carte, e_color='green', e_name='edges hors autoroute', n_name='nodes hors autoroute', n_color='black', **param_exp_gr)
carte = dgr_autoroute.explore(refmap=carte, n_color='black', e_name='edges autoroute', n_name='nodes autoroute', **param_exp_gr)
carte = dgr_peu_maille.explore(refmap=carte, **param_peu_maille, nodes=False)
carte = dgr_stat_indispo.explore(refmap=carte, **param_non_maille)
carte = dgr_non_maille.explore(refmap=carte, layercontrol=True, nodes=False, **param_non_maille)
carte

## Indicateur AFIR

In [ ]:
longueur_rtet = sum([gr_tot.edges[edge]['weight'] for edge in rtet_edges])
longueur_non_afir = sum([gr_non_maille.edges[edge]['weight'] for edge in gr_non_maille.edges])
longueur_peu_afir = sum([gr_peu_maille.edges[edge]['weight'] for edge in gr_peu_maille.edges])
print("pourcentage atteint : ", (1 - longueur_non_afir/longueur_rtet) * 100, (1 - longueur_peu_afir/longueur_rtet) * 100)
print("distance totale, restante : ", longueur_rtet / 1000, longueur_non_afir / 1000, longueur_peu_afir / 1000)

In [ ]:
len([gr_peu_maille.edges[edge]['weight'] for edge in gr_peu_maille.edges]), len([dgr_peu_maille.edges[edge]['weight'] for edge in dgr_peu_maille.edges])

In [ ]:
longueur_rtet = sum([dgr_tot.edges[edge]['weight'] for edge in drtet_edges])
longueur_non_afir = sum([dgr_non_maille.edges[edge]['weight'] for edge in dgr_non_maille.edges])
longueur_peu_afir = sum([dgr_peu_maille.edges[edge]['weight'] for edge in dgr_peu_maille.edges])
print("pourcentage atteint : ", (1 - longueur_non_afir/longueur_rtet) * 100, (1 - longueur_peu_afir/longueur_rtet) * 100)
print("distance totale, restante : ", longueur_rtet / 1000 / 2, longueur_non_afir / 1000 / 2, longueur_peu_afir / 1000 / 2)